In [15]:
import pandas as pd
df_bert = pd.read_csv('./data/raw/Education_videos_7_classified_BERT_batch_0_10000.csv')
df_bart = pd.read_csv('./data/raw/Education_videos_7_classified_BART_batch_0_10000.csv')

In [16]:
import re
import ast
def convert_column_to_list(df, column_name):
    df[column_name] = df[column_name].apply(lambda x: re.findall(r"'(.*?)'", x))
    df[column_name] = df[column_name].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    return df

df_bert = convert_column_to_list(df_bert, 'classified_labels')
df_bart = convert_column_to_list(df_bart, 'classified_labels')

In [22]:
number_equal = (df_bert['classified_labels'] == df_bart['classified_labels']).sum()
print(number_equal)

1293


In [25]:
def compare_lists(df1, df2, column_name):
    exact_matches = 0
    partial_matches = 0
    total_rows = min(len(df1), len(df2))

    for i in range(total_rows):
        list1 = set(df1.at[i, column_name])
        list2 = set(df2.at[i, column_name])

        if list1 == list2:
            exact_matches += 1
        elif list1 & list2:
            partial_matches += 1

    return exact_matches, partial_matches

exact, partial = compare_lists(df_bert, df_bart, 'classified_labels')

In [27]:
print(exact)
print(partial)

1319
1915


In [108]:
import numpy as np
def sample_mismatched_rows(df1, df2, column_name, sample_size=10):
    mismatched_rows = []
    total_rows = min(len(df1), len(df2))

    for i in range(total_rows):
        set1 = set(df1.at[i, column_name])
        set2 = set(df2.at[i, column_name])
        
        # If sets do not match, add to mismatched_rows
        if ((set1 != set2) or not (set1 & set2)):
            mismatched_rows.append((i, df1.at[i, column_name], df2.at[i, column_name]))
    
    # Return a sample of mismatched rows
    return mismatched_rows

# Get a sample of mismatched rows
mismatched_rows = sample_mismatched_rows(df_bert, df_bart, 'classified_labels', sample_size=10)
mismatched_rows = pd.DataFrame(mismatched_rows)
mismatched_rows.columns = [['index', 'BERT', 'BART']]

def first_element(row):
    return row[0][0]

def second_element(row) :
    list = row[0]
    if len(list) > 1:
        return list[1]
    else :
        return None
    
def third_element(row) :
    list = row[0]
    if len(list) > 2 :
        return list[2]
    else :
        return None

mismatched_rows['first_element'] = mismatched_rows['BART'].apply(first_element, axis=1)
mismatched_rows['second_element'] = mismatched_rows['BART'].apply(second_element, axis=1)
mismatched_rows['third_element'] = mismatched_rows['BART'].apply(third_element, axis=1)
mismatched_rows

C:\Users\fpili\AppData\Local\Temp\ipykernel_18772\1768481728.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return row[0][0]
C:\Users\fpili\AppData\Local\Temp\ipykernel_18772\1768481728.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  list = row[0]
C:\Users\fpili\AppData\Local\Temp\ipykernel_18772\1768481728.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  list = row[0]


,index,BERT,BART,first_element,second_element,third_element
0,0,[conference],[misc],misc,None,None
1,1,[tutorial or DIY],[misc],misc,None,None
2,2,[tutorial or DIY],[misc],misc,None,None
3,4,[tutorial or DIY],[misc],misc,None,None
4,6,[tutorial or DIY],[misc],misc,None,None
...,...,...,...,...,...,...
8676,9994,[kids content],"[kids content, tutorial or DIY]",kids content,tutorial or DIY,None
8677,9995,[kids content],"[kids content, entertaining explanation or sci...",kids content,entertaining explanation or science populariza...,tutorial or DIY
8678,9996,[entertaining explanation or science populariz...,[kids content],kids content,None,None
8679,9997,"[kids content, conference]",[kids content],kids content,None,None
